In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

In [2]:
def undiv_dif_3(x, y, z):
    return abs(x - 2 * y + z)
def calc_U_from_W(W, gamma):
    return np.array([W[0], W[0] * W[1], 0.5 * W[0] * W[1] ** 2 + W[2] / (gamma - 1)])
def calc_res_W_from_res_U(U, gamma):
    res_W = np.zeros_like(U)
    for i in range(0, res_W.shape[0]):
        for j in range(0, res_W.shape[1]):
            res_W[i][j][0] = U[i][j][0]
            res_W[i][j][1] = U[i][j][1] / U[i][j][0]
            res_W[i][j][2] = (gamma - 1) * (U[i][j][2] - 0.5 * U[i][j][1] ** 2 / U[i][j][0])
    return res_W
def calc_F_from_U(U, gamma):
    p = (gamma - 1) * (U[2] - 0.5 * U[1] ** 2 / U[0])
    return np.array([U[1], U[1] ** 2 / U[0] +  p, (U[2] + p) * U[1] / U[0]])
#Evaluating U on the boundaries using second order ENO scheme
def eval_u_cell_bnd_ENO(u):
    res = np.zeros((np.shape(u)[0], 2))
    res[0][0] = (u[0] + u[1]) / 2
    res[1][0] = (u[1] + u[2]) / 2
    res[1][1] = (3 * u[1] - u[2]) / 2
    res[np.shape(res)[0] - 1][1] = (u[np.shape(res)[0] - 1] + u[np.shape(res)[0] - 2]) / 2
    res[np.shape(res)[0] - 2][0] = (u[np.shape(res)[0] - 2] + u[np.shape(res)[0] - 1]) / 2
    res[np.shape(res)[0] - 2][1] = (3 * u[np.shape(res)[0] - 2] - u[np.shape(res)[0] - 1]) / 2
    for i in range(2, np.shape(res)[0] - 2):
        d_1_l = abs(u[i] - u[i - 1])
        d_1_r = abs(u[i + 1] - u[i])
        if (d_1_r < d_1_l):
            if undiv_dif_3(u[i - 1], u[i], u[i + 1]) < undiv_dif_3(u[i], u[i + 1], u[i + 2]):
                res[i][0] = (-1 / 6 * u[i - 1] + 5 / 6 * u[i] + 1 / 3 * u[i + 1])
                res[i][1] = (1 / 3 * u[i - 1] + 5 / 6 * u[i] - 1 / 6 * u[i + 1])
            else:
                res[i][0] = (1 / 3 * u[i] + 5 / 6 * u[i + 1] - 1 / 6 * u[i + 2])
                res[i][1] = (11 / 6 * u[i] - 7 / 6 * u[i + 1] + 1 / 3 * u[i + 2])
        else:
            if undiv_dif_3(u[i - 1], u[i], u[i + 1]) < undiv_dif_3(u[i - 2], u[i - 1], u[i]):
                res[i][0] = (-1 / 6 * u[i - 1] + 5 / 6 * u[i] + 1 / 3 * u[i + 1])
                res[i][1] = (1 / 3 * u[i - 1] + 5 / 6 * u[i] - 1 / 6 * u[i + 1])
            else:
                res[i][0] = (1 / 3 * u[i - 2] - 7 / 6 * u[i - 1] + 11 / 6 * u[i])
                res[i][1] = (- 1 / 6 * u[i - 2] + 5 / 6 * u[i - 1] + 1 / 3 * u[i])
    return res
#Evaluating flux through the boundary using HLL Riemann Solver
def eval_F(U, gamma):
    res = np.zeros((np.shape(U)[0] - 1, 3))
    cell_bnd_1 = eval_u_cell_bnd_ENO(U[:, 0])
    cell_bnd_2 = eval_u_cell_bnd_ENO(U[:, 1])
    cell_bnd_3 = eval_u_cell_bnd_ENO(U[:, 2])
    for i in range(0, np.shape(res)[0]):
        u_ = ( U[i, 1] / (U[i, 0] ** 0.5) + U[i + 1, 1] / (U[i + 1, 0] ** 0.5) ) / ( (U[i, 0] ** 0.5) + (U[i + 1, 0] ** 0.5 ))
        H_L = (U[i, 2] + (gamma - 1) * (U[i, 2] - 0.5 * U[i, 1] ** 2 / U[i, 0])) / U[i, 0]
        H_R = (U[i + 1, 2] + (gamma - 1) * (U[i + 1, 2] - 0.5 * U[i + 1, 1] ** 2 / U[i + 1, 0])) / U[i + 1, 0]
        H_ = ((U[i, 0] ** 0.5) * H_L + (U[i + 1, 0] ** 0.5) * H_R) / ((U[i, 0] ** 0.5) + (U[i + 1, 0] ** 0.5))
        a_ = ((gamma - 1) * (H_ - 0.5 * u_ ** 2)) ** 0.5
        S_L = u_ - a_
        S_R = u_ + a_
        U_L_ = np.array([cell_bnd_1[i, 0], cell_bnd_2[i, 0], cell_bnd_3[i, 0]])
        U_R_ = np.array([cell_bnd_1[i + 1, 1], cell_bnd_2[i + 1, 1], cell_bnd_3[i + 1, 1]])
        F_L = calc_F_from_U(U_L_, gamma)
        F_R = calc_F_from_U(U_R_, gamma)
        if S_L >= 0:
            res[i] = F_L.copy()
        elif S_L <= 0 and 0 <= S_R:
            res[i] = (S_R * F_L - S_L * F_R + S_L * S_R * (U_R_ - U_L_)) / (S_R - S_L)
        else:
            res[i] = F_R.copy()
    return res



In [3]:
#Calculations for rarefaction wave 
def C_A(x, t, W_L, gamma):
    C_L = (gamma * W_L[2] / W_L[0]) ** 0.5
    return (gamma - 1) * (W_L[1] + 2 * C_L / (gamma - 1) - x / t) / (gamma + 1)
#Get exact solution for this specific problem and initial state. In this case - Sod test problem
def get_exact_solution(M, N, tau, h):
    W_L = np.array([1.0, 0.75, 1.0]) 
    W_R = np.array([0.125, 0.0, 0.1])
    C_L = (gamma * W_L[2] / W_L[0]) ** 0.5
    W_C = np.array([0, 1.3609, 0.466294])
    W_C[0] = W_L[0] * (W_C[2] / W_L[2]) ** (1 / gamma) 
    u_1 = W_L[1] - (gamma * W_L[2] / W_L[0]) ** 0.5 
    u_2 = W_C[1] - (gamma * W_C[2] / W_C[0]) ** 0.5
    u_3 = W_C[1]
    u_4 = W_R[1] + (0.5 * W_R[0] * ((gamma + 1) * W_C[2] + (gamma - 1) * W_R[2])) ** 0.5 / W_R[0]
    rho_w = W_R[0] * (W_R[1] - u_4) / (W_C[1] - u_4)
    sol = np.zeros((M, N, 3))
    for n in range(0, N):
        if n * h < 0.5:
            sol[0][n][:] = W_L.copy()
        else:
            sol[0][n][:] = W_R.copy()
    for m in range(1, M):
        t = m * tau
        for n in range(0, N):
            x = n * h
            if x < 0.5 + u_1 * t:
                sol[m][n][:] = W_L.copy()
            elif x < 0.5 + u_2 * t:
                sol[m][n][0] = W_L[0] * (C_A(x - 0.5, t, W_L, gamma) / C_L) ** (2 / (gamma - 1))
                sol[m][n][1] = C_A(x - 0.5 , t, W_L, gamma) + (x - 0.5)/ t
                sol[m][n][2] = W_L[2] * (C_A(x - 0.5, t, W_L, gamma) / C_L) ** (2 * gamma / (gamma - 1))
            elif x < 0.5 + u_3 * t:
                sol[m][n][:] = W_C.copy()
            elif x < 0.5 + u_4 * t:
                sol[m][n][:] = W_C.copy()
                sol[m][n][0] = rho_w
            else:
                sol[m][n][:] = W_R.copy()
    return sol

In [4]:
#Initial state
h = 0.01
N = int(1.0 / h + 1)
CFL = 0.1
tau = CFL * h
T = 0.2
M = int(T / tau) + 1
W_L = np.array([1.0, 0.75, 1.0]) 
W_R = np.array([0.125, 0.0, 0.1]) 
gamma = 1.4
U_L = calc_U_from_W(W_L, gamma)
U_R = calc_U_from_W(W_R, gamma)
res = np.zeros((M, N, 3))
for n in range(0, N):
    if n * h < 0.5:
        res[0, n, :] = U_L.copy()
    else:
        res[0, n, :] = U_R.copy()
#Time loop: explicit Euler method
for m in range(1, M):
    res[m, 0, :] = U_L.copy()
    res[m, N - 1, :] = U_R.copy()
    F = eval_F(res[m - 1, :, :], gamma)
    for n in range(1, N - 1):
        res[m, n, :] = res[m - 1, n, :].copy()
        res[m, n, :] -= CFL * (F[n] - F[n - 1]) 
res_W = calc_res_W_from_res_U(res, gamma)
exact_W = get_exact_solution(M, N, tau, h)

In [5]:
#Visualize density
df = pd.DataFrame()
df["x"] = list(np.linspace(0, 1, N)) * 2 * M
df["t"] =  np.append(np.reshape( np.transpose( np.full( (N, M), np.linspace(0, T, M) ) ), (M * N, 1) ), np.reshape( np.transpose( np.full( (N, M), np.linspace(0, T, M) ) ), (M * N, 1) ) )
df["y"] =  np.append(np.reshape(exact_W[:, :, 0], (N * M, 1)), np.reshape(res_W[:, :, 0], (N * M, 1)))
df["color"] = [0.5] * N * M + [2.5] * N * M
fig = px.scatter(df, x="x", y="y", animation_frame="t", color="color", color_continuous_scale=px.colors.sequential.Viridis, range_color=[0, 4], range_y=[-0.03, 1.03], title="Density")
fig.update_layout(coloraxis_showscale=False)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 5
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.show()

In [6]:
#Visualize velocity
df = pd.DataFrame()
df["x"] = list(np.linspace(0, 1, N)) * 2 * M
df["t"] =  np.append(np.reshape( np.transpose( np.full( (N, M), np.linspace(0, T, M) ) ), (M * N, 1) ), np.reshape( np.transpose( np.full( (N, M), np.linspace(0, T, M) ) ), (M * N, 1) ) )
df["y"] =  np.append(np.reshape(exact_W[:, :, 1], (N * M, 1)), np.reshape(res_W[:, :, 1], (N * M, 1)))
df["color"] = [0.5] * N * M + [2.5] * N * M
fig = px.scatter(df, x="x", y="y", animation_frame="t", color="color", color_continuous_scale=px.colors.sequential.Viridis, range_color=[0, 4], range_y=[-0.03, 1.7], title="Velocity")
fig.update_layout(coloraxis_showscale=False)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 5
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.show()

In [7]:
#Visualize pressure
df = pd.DataFrame()
df["x"] = list(np.linspace(0, 1, N)) * 2 * M
df["t"] =  np.append(np.reshape( np.transpose( np.full( (N, M), np.linspace(0, T, M) ) ), (M * N, 1) ), np.reshape( np.transpose( np.full( (N, M), np.linspace(0, T, M) ) ), (M * N, 1) ) )
df["y"] =  np.append(np.reshape(exact_W[:, :, 2], (N * M, 1)), np.reshape(res_W[:, :, 2], (N * M, 1)))
df["color"] = [0.5] * N * M + [2.5] * N * M
fig = px.scatter(df, x="x", y="y", animation_frame="t", color="color", color_continuous_scale=px.colors.sequential.Viridis, range_color=[0, 4], range_y=[-0.03, 1.03], title="Pressure")
fig.update_layout(coloraxis_showscale=False)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 5
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.show()

In [58]:
#Visualize density
df = pd.DataFrame()
df["x"] = list(np.linspace(0, 1, N)) * 2
df["y"] =  np.append(exact_W[M - 1, :, 0], res_W[M - 1, :, 0])
df["color"] = [1] * N + [2] * N
fig = px.line(df, x="x", y="y", color="color",  range_y=[-0.03, 1.03], title="Density", color_discrete_map={1:"navy", 2:"royalblue"})
fig.update_layout(showlegend=False)
fig.write_image("./density.png", engine="kaleido", width=1000, height=600)
fig.show()

In [59]:
#Visualize velocity
df = pd.DataFrame()
df["x"] = list(np.linspace(0, 1, N)) * 2
df["y"] =  np.append(exact_W[M - 1, :, 1], res_W[M - 1, :, 1])
df["color"] = [1] * N + [2] * N
fig = px.line(df, x="x", y="y", color="color", range_y=[-0.03, 1.7], title="Velocity", color_discrete_map={1:"navy", 2:"royalblue"})
fig.update_layout(showlegend=False)
fig.write_image("./velocity.png", engine="kaleido", width=1000, height=600)
fig.show()

In [60]:
#Visualize pressure
df = pd.DataFrame()
df["x"] = list(np.linspace(0, 1, N)) * 2
df["y"] =  np.append(exact_W[M - 1, :, 2], res_W[M - 1, :, 2])
df["color"] = [1] * N + [2] * N
fig = px.line(df, x="x", y="y", color="color", range_y=[-0.03, 1.03], title="Pressure", color_discrete_map={1:"navy", 2:"royalblue"})
fig.update_layout(showlegend=False)
fig.write_image("./pressure.png", engine="kaleido", width=1000, height=600)
fig.show()